Here we want to develop code to perform bespoke deduplication using the output from ``umi_tools group``

In [1]:
import pysam
import collections
import copy
from time import gmtime, strftime

In [9]:
infile = "/home/FILESERVER5/proteomics/tss38/git_repos/cell_barcode/run/mapped/hgmm_100_hg_mm_grouped.bam"

In [417]:
def makeFastqEntryFromBAMRead(read):
    read_id = read.query_name
    seq = read.seq
    qualities = read.qual
    return ("\n".join([read_id, seq, "+", qualities]))
        
    
    
read_ids = set()
gene2reads = collections.defaultdict(set)
group2reads = collections.defaultdict(set)
gene2groups = collections.defaultdict(set)
cell2groups = collections.defaultdict(set)
cell2reads = collections.defaultdict(set)
read2groups = collections.defaultdict(lambda: collections.defaultdict(set))

n = 0
for read in pysam.Samfile(infile, "rb").fetch(until_eof=True):
    if n % 1000000 == 0:
        print(n, strftime("%Y-%m-%d %H:%M:%S", gmtime()))
    n+=1
    read_name = read.query_name
    read_id, cell, umi = read_name.split("_")
    
    # remove the redundant first three sections to save space
    short_read_id = ":".join(read_id.split(":")[3:]) 
    
    try:
        gene = read.get_tag("XT")
        group = read.get_tag("UG")
        
    except:
        continue # no gene assigned

    read_ids.add(short_read_id)
    gene2reads[gene].add(short_read_id)
    group2reads[group].add(short_read_id)
    gene2groups[gene].add(group)
    cell2groups[cell].add(group)
    cell2reads[cell].add(short_read_id)
    read2groups[cell][short_read_id].add(group)
    
print(n, strftime("%Y-%m-%d %H:%M:%S", gmtime()))

0 2017-08-23 13:34:39
1000000 2017-08-23 13:34:58
2000000 2017-08-23 13:35:09
3000000 2017-08-23 13:35:21
4000000 2017-08-23 13:35:34
4359149 2017-08-23 13:35:36


In [370]:
print(len(read_ids))
print(len(gene2reads))
print(len(group2reads))
print(len(gene2groups))
print(len(cell2groups))

4204636
24323
3091570
24323
100


In [441]:
# we want to find the minimum set of UMI groups which will account for all reads

n_cell = 0
cell2groups_output_one_read = collections.defaultdict(set)
cell2reads2output = collections.defaultdict(set)
for cell in cell2reads:
    accounted_reads = set()
    #print(cell)
    #print("reads: %s" % len(cell2reads[cell]))
    #print("groups: %s" % len(cell2groups[cell]))
    #for cell in ["ATGAGGGAGTAGTGCG"]:
    #print("Looking for single gene->group matches")
    retain_groups = set()
    single_group_reads = set()
    for read in cell2reads[cell]:
        if len(read2groups[cell][read])==1:
            for group in read2groups[cell][read]:
                if len(group2reads[group]) == 1:
                    retain_groups.add(group)
            accounted_reads.add(read)
            single_group_reads.add(read)

    intersecting_groups = cell2groups[cell].difference(retain_groups)
    intersecting_group_reads = {x:group2reads[x].difference(accounted_reads) for x in intersecting_groups}

    #print("accounted reads: %s" % len(accounted_reads))
    #print("retained group: %s" %  len(retain_groups))
    
    #print("Looking for minimum set of groups to cover remaining reads")
    
    n = 0
    multi_read_groups = set()


    looking_n = 0
    while len(cell2reads[cell].difference(accounted_reads)) > 0:
        
        intersecting_groups = sorted(intersecting_groups, reverse=True,
                                     key=lambda x: len(intersecting_group_reads[x]))
    
        if looking_n % 100 == 0:
            pass
            #print("Still looking for groups for %s reads" % len(cell2reads[cell].difference(accounted_reads)))
        
        looking_n += 1
        
        try:
            top_group = intersecting_groups[0]
        except:
            print(len(accounted_reads), len(cell2reads[cell]))
            print(intersecting_groups)
            raise ValueError()
        
        top_reads = intersecting_group_reads[top_group]
        retain_groups.add(top_group)
        multi_read_groups.add(top_group)
        accounted_reads.update(top_reads)
        intersecting_groups = intersecting_groups[1:] # remove top group
        
        for group in intersecting_groups:
            # update counts to only unaccounted reads
            intersecting_group_reads[group] = intersecting_group_reads[group].difference(top_reads)
    
    #print("Finished looking for minimum set of groups to cover remaining reads")
    #print("groups retained %s" % len(retain_groups))
    
    #print("Finished looking for minimum set of groups to cover remaining reads")
    multi_read_group_reads = set()

    for group in multi_read_groups:
        multi_read_group_reads.update(group2reads[group].difference(single_group_reads))

    multi_read_group_reads_groups = {x:read2groups[cell][x].intersection(multi_read_groups)
                                     for x in multi_read_group_reads}

    groups_covered = set()
    accounted_multi_read_groups = set()
    keep_multi_reads = set()
    while accounted_multi_read_groups!= multi_read_groups:

        multi_read_group_reads = sorted(multi_read_group_reads, reverse=True,
                                        key=lambda x: len(multi_read_group_reads_groups[x]))

        top_read = multi_read_group_reads[0]
        keep_multi_reads.add(top_read)

        groups_covered.update(read2groups[cell][top_read])

        top_groups = multi_read_group_reads_groups[top_read].intersection(multi_read_groups)
        accounted_multi_read_groups.update(top_groups)
        multi_read_group_reads = multi_read_group_reads[1:]

        for read in multi_read_group_reads_groups:
            multi_read_group_reads_groups[read] = multi_read_group_reads_groups[read].difference(top_groups)
    
    cell2reads2output[cell] = keep_multi_reads
    cell2groups_output_one_read[cell] = cell2groups[cell].difference(groups_covered)
    
    if n_cell % 10 == 0:
        print("processed %s cells" % n_cell, strftime("%Y-%m-%d %H:%M:%S", gmtime()))
    n_cell += 1

411681 {'1:1113:5122:8787', '4:1225:13179:5974', '4:1225:13169:5956', '5:2201:19055:20797'}
processed 0 cells 2017-08-23 14:42:48
2424869 {'2:1219:18020:21184', '7:2213:12733:45537'}
2863135 {'1:1223:16508:35725', '6:2112:15240:20586'}
455707 {'8:1208:25215:24331', '8:1208:25266:48720'}
2132051 {'1:1101:9039:26265', '6:2122:14346:34090', '6:2123:13068:26599'}
2005004 {'1:2105:28270:32261', '3:1212:23206:4532', '8:1126:32319:5939', '7:2226:30279:19742'}
639008 {'4:2114:20983:5780', '6:2118:8115:47981', '2:1227:10612:10387'}
563265 {'3:1201:19624:19636', '7:2203:3965:47120', '3:1201:19126:19619', '7:1204:6766:12304', '4:2205:31101:36675'}
1087495 {'3:2206:7456:22713', '6:1207:7984:25527'}
2384021 {'4:2225:2392:11513', '2:1117:29031:16594'}
3055616 {'6:2212:25520:27004', '5:2126:14407:37748'}
processed 10 cells 2017-08-23 14:43:34
370690 {'8:2123:5528:29114', '6:2218:18710:12638'}
2596923 {'3:1205:26555:3723', '8:2109:23997:15012', '1:2227:23216:42425'}
385024 {'6:1209:27336:7293', '3:211

In [442]:
print(cell)

CTTGGCTCAGGGAGAG


In [376]:
print(len(cell2reads2output[cell]))
print(len(cell2groups_output_one_read[cell]))

707
55165


In [415]:
n = 1
last_group = None
reads = []
for read in pysam.Samfile(infile, "rb").fetch(until_eof=True):

    if n % 1000000 == 0:
        print(n, strftime("%Y-%m-%d %H:%M:%S", gmtime()))

    read_name = read.query_name
    read_id, cell, umi = read_name.split("_")
    
    #if not cell == "ATGAGGGAGTAGTGCG":
    #    continue
    
    try:
        group = read.get_tag("UG")
        
    except:
        continue # no gene assigned
    

    if last_group and (group != last_group):
        if group in cell2groups_output_one_read[cell]:
            output_read = None
            min_nh = 100
            for read in reads:
                nh = read.get_tag("NH")
                if nh < min_nh:
                    min_nh = nh
                    output_read = read
                    
            assert output_read is not None, "this shouldn't happen!"
            print(makeFastqEntryFromBAMRead(read))
        else:
            for read in reads:
                read_name = read.query_name
                read_id, cell, umi = read_name.split("_")
                short_read_id = ":".join(read_id.split(":")[3:]) 
                if short_read_id in cell2reads2output[cell]:
                    print(makeFastqEntryFromBAMRead(read))
                    cell2reads2output[cell].discard(short_read_id)
                
            print(group, group in multi_read_groups)

        reads = [read]
        #break
        n+=1
    
    else:
        reads.append(read)
    
    if n>10:
        print(n)
        break
        
    last_group = group

print(n, strftime("%Y-%m-%d %H:%M:%S", gmtime()))

ST-K00126:308:HFLYFBBXX:1:1206:14204:39436_ATGAGGGAGTAGTGCG_GTAAATATTG
TGGGATAGTGTATGCATTCGCTGCAGGGTATTTAGAAATGATGGTGTCCAATCGGGCGCACATTGAGGGAAACATGCAAGTTCCTTTCTTGGACAATG
+
-JJJFAFJFJFF<FF<-JAJA7JJFJJFFJJJJJJFAFJFJFFJJJ<JFJFJJJJJJJFFJFJAJJJJJFJJA<7JJFJFJAJJJJJJJJFAFFFA<A
ST-K00126:308:HFLYFBBXX:4:2122:15534:27672_ATGAGGGAGTAGTGCG_GGGATCGATA
GTAGCTGTGAGAGGTTACTCATGAAAGTAGGTAGCCTAGTAAATTAGTGGTAAGTCAGGGGAAGGGAATCAGCCTCCCGTTCCATTTCATTCCCTTCC
+
<JFF<FFF<FFAAAA-7<FAF7FJFJAA<JJFF-F7A<7JJJAA7JJJJJJJFA-AJAF<JJAF7777--A<7-7-77-7-JJFJJAFJFFF<A-<<-
ST-K00126:308:HFLYFBBXX:3:1103:11992:44500_ATGAGGGAGTAGTGCG_CCACGCGTTG
CCAGTTTGCTTGATTTGTTGAGCCACCATCCCTATGAATACACCTTACATCTCTTTCAAATGTGGAGATGGTTTTTTGATCTTACAGTGTGTGGAACC
+
AAF-JFFAAJJJJJJJFJJFJFAJJJJFFJ7JJJJJJJJFJJJJFFFJJJJFJJJJJJFJJJJJJJJJJJJJJJFJAFJFJFJJJFFJJAFAFA<7<A
ST-K00126:308:HFLYFBBXX:1:1216:27955:47471_ATGAGGGAGTAGTGCG_AGGAATGTCC
TGACATAGCTTAGGACACTTGCTCATTTATTTAAAAGAGTCCCTTCACACAGTGCAGATAAAAGAAGGCAAGAGCATAGGATTGTGGCATGTGCTGGG
+
FJJJJJFJJJJJJJJ

/home/FILESERVER5/proteomics/tss38/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:33: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.


In [214]:
print(group2reads[171616])
print(group2reads[171809])
for cell in cell2groups:
    for gene in cell2groups[cell]:
        if 171616 in cell2groups[cell][gene] or 171809 in cell2groups[cell][gene]:
            print(cell, gene)
            print(cell2groups[cell][gene], cell2reads[cell][gene])

{'6:1207:17442:15223', '3:1104:26778:37343'}
{'6:1207:17442:15223', '3:1104:26778:37343'}
ATGAGGGAGTAGTGCG ENSG00000269713
{171616} {'6:1207:17442:15223', '3:1104:26778:37343'}
ATGAGGGAGTAGTGCG ENSG00000271425
{171809} {'6:1207:17442:15223', '3:1104:26778:37343'}


In [188]:
print(cell)
print(sum([len(cell2groups[cell][x]) for x in cell2groups[cell]]))
print(len(cell_groups))

ATGAGGGAGTAGTGCG
56691
56691


In [166]:
print(len(group2reads))
print(len(keep_groups))
discard_groups = set(group2reads).difference(keep_groups)
print(len(discard_groups))

3091570
3091570
0


In [30]:
keep_reads = set()
#for cell in cell2groups:
print(cell)
for gene in cell2groups[cell]:
    accounted_groups = set()
    while len(accounted_groups) != len()

3091570
926567


KeyboardInterrupt: 

In [444]:
import CGAT.Fastq as Fastq

In [463]:
infile = "/home/FILESERVER5/proteomics/tss38/git_repos/cell_barcode/run/mapped/test.fastq"
reads = set()
for entry in Fastq.iterate(open(infile)):
    if entry.identifier not in reads:
        reads.add(entry.identifier)
    else:
        print(entry.identifier, entry.identifier in reads)

print(entry)
    

@ST-K00126:308:HFLYFBBXX:4:2126:17665:10053_TTTATGCAGTTGTCGT_GGCATTGTGT
GCCCCGCTGGACGGACGGACGGACGCACGCACGCCGTCAGGTGCGCGTGTCGCCGAGGCCGCCAGGACGGAGCGATTCTCACGGAGGAAGGAGCACGC
+
<<-7A7<A<AJFJFJJFFJJJJJJJAJJFJJJFJJJJJJJJFJJJJJFJJJJJJJJJJJJFJJJJJJJJJJJJJJ77FFJJJJFFJJJJJFFJJFJ<<
